In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Liste pour stocker toutes les données
toutes_donnees = []

# Boucler sur les 5 pages
for page in range(1, 6):
    print(f"📄 Traitement de la page {page}...")

    # URL de la page courante
    url_page = f"https://www.tayara.tn/ads/c/Véhicules/Voitures/?page={page}"

    # Requête pour la page liste
    response = requests.get(url_page)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver toutes les annonces
    annonces = soup.find_all('article')

    print(f"   🔍 {len(annonces)} annonces trouvées")

    # Traiter chaque annonce
    for i, annonce in enumerate(annonces):
        try:
            # Récupérer le lien vers la page détaillée
            lien = annonce.find('a')['href']
            url_detail = "https://www.tayara.tn" + lien #concaténation pour accéder à la voiture sélectionnée

            # Aller sur la page détaillée
            response_detail = requests.get(url_detail)
            soup_detail = BeautifulSoup(response_detail.content, 'html.parser')
            #html.parser comprend le code HTML et le transforme en arbre compréhensible par python

            # Extraire le titre et prix de base
            titre = annonce.find('h2')
            prix = annonce.find('data') #le prix dans balise data

            # Dictionnaire pour cette annonce
            donnees_annonce = {
                'Titre': titre.text.strip() if titre else 'N/A',
                'Prix': prix.text.strip() if prix else 'N/A',
                'Page': page,
                'URL': url_detail
            }

            # EXTRAIRE LES 11 CARACTÉRISTIQUES
            features = soup_detail.find_all('li', class_='col-span-6') #features de voiture sous forme de li - on peut trouver 11 au max (10 possible ....)

            for feature in features:
                label = feature.find('span', class_='text-gray-600/80') #label titre feature (couleur, kilométrage ...)
                valeur = feature.find('span', class_='text-gray-700/80') #valeur (polo, rouge, essence ....)

                if label and valeur:
                    donnees_annonce[label.text.strip()] = valeur.text.strip() #supprimer les espaces inutiles

            # Ajouter aux données
            toutes_donnees.append(donnees_annonce)
            print(f"   ✅ Annonce {i+1}: {len(features)} caractéristiques")

            # Pause pour éviter de surcharger le site
            time.sleep(1)

        except Exception as e:
            print(f"Erreur annonce {i+1}: {e}")
            continue

    print(f"Page {page} terminée\n")


# Sauvegarder toutes les données en CSV
if toutes_donnees:
    with open('tayara_5_pages.csv', 'w', newline='', encoding='utf-8') as f:
        # Obtenir toutes les colonnes
        colonnes = set() #set() ne respecte pas l ordre

        # Parcourir CHAQUE annonce pour trouver TOUTES les colonnes possibles
        for annonce in toutes_donnees:
            colonnes.update(annonce.keys())  # Ajoute les clés de cette annonce

        print("Colonnes trouvées:", colonnes)
        # → {'Page', 'Titre', 'Prix', 'URL', 'Puissance fiscale', 'Type carrosserie', 'Kilométrage', ...}

        # Ordonner : d'abord les colonnes fixes, puis les caractéristiques
        colonnes_fixes = ['Page', 'Titre', 'Prix', 'URL']
        colonnes_caracteristiques = [c for c in colonnes if c not in colonnes_fixes]

        colonnes_ordonnees = colonnes_fixes + colonnes_caracteristiques

        #Crée un objet qui sait écrire des dictionnaires dans un fichier CSV, avec cet ordre de colonnes
        writer = csv.DictWriter(f, fieldnames=colonnes_ordonnees)
        #Écris la première ligne du CSV avec les noms des colonnes
        writer.writeheader()
        #Écris toutes les lignes de données d'un coup
        writer.writerows(toutes_donnees)

    print(f"🎉 TERMINÉ !")
    print(f"📊 {len(toutes_donnees)} annonces sauvegardées")
    print(f"📁 Fichier: tayara_5_pages.csv")
else:
    print("❌ Aucune donnée trouvée")

📄 Traitement de la page 1...
   🔍 46 annonces trouvées
   ✅ Annonce 1: 11 caractéristiques
   ✅ Annonce 2: 11 caractéristiques
   ✅ Annonce 3: 10 caractéristiques
   ✅ Annonce 4: 11 caractéristiques
   ✅ Annonce 5: 11 caractéristiques
   ✅ Annonce 6: 11 caractéristiques
   ✅ Annonce 7: 5 caractéristiques
   ✅ Annonce 8: 11 caractéristiques
   ✅ Annonce 9: 11 caractéristiques
   ✅ Annonce 10: 9 caractéristiques
   ✅ Annonce 11: 11 caractéristiques
   ✅ Annonce 12: 11 caractéristiques
   ✅ Annonce 13: 11 caractéristiques
   ✅ Annonce 14: 11 caractéristiques
   ✅ Annonce 15: 11 caractéristiques
   ✅ Annonce 16: 11 caractéristiques
   ✅ Annonce 17: 11 caractéristiques
   ✅ Annonce 18: 11 caractéristiques
   ✅ Annonce 19: 11 caractéristiques
   ✅ Annonce 20: 11 caractéristiques
   ✅ Annonce 21: 11 caractéristiques
   ✅ Annonce 22: 11 caractéristiques
   ✅ Annonce 23: 11 caractéristiques
   ✅ Annonce 24: 11 caractéristiques
   ✅ Annonce 25: 11 caractéristiques
   ✅ Annonce 26: 11 caractérist

Le code utilise un set() parce qu'on ne sait pas à l'avance :

Quelles caractéristiques chaque voiture aura

Combien de caractéristiques exactement

L'ordre des caractéristiques

La méthode avec set() est plus flexible et s'adapte automatiquement aux données trouvées !